In [2]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import ast
import json

In [128]:
bbox_df = pd.read_csv("bbox_df.csv")

In [129]:
bbox_df["image_file"] = bbox_df["image_file"].astype(str).apply(lambda x: x.rjust(12, '0') + ".jpg")
bbox_df['bbox'] = bbox_df['bbox'].apply(ast.literal_eval)

In [130]:
bbox_df['image_file'] = str('val2017/')+bbox_df['image_file']

In [ ]:
bbox_df.head()

In [208]:
# 엔트로피 계산
def calculate_image_entropy_(image):
    # image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    # 이미지 히스토그램 계산
    hist, _ = np.histogram(image.flatten(), bins=256, range=[0,256])

    # 확률 계산
    prob = hist / np.sum(hist)

    # 엔트로피 계산
    entropy = -np.sum(prob * np.log2(prob + 1e-10))  # 1e-10은 로그 계산 시 분모가 0이 되는 것을 방지하기 위한 작은 값

    return entropy

In [217]:
def calculate_laplacian_variance_(image):
    # 이미지 읽기 (흑백으로 변환)
    # image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 라플라시안 필터 적용
    laplacian = cv2.Laplacian(image, cv2.CV_64F)

    # 분산 계산
    variance_of_laplacian = laplacian.var()

    return variance_of_laplacian

In [218]:
entropy = []
laplacian = []

for i in bbox_df['bbox_id']:
    bbox = bbox_df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = bbox_df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    entropy_value = calculate_image_entropy_(cropped_image)
    laplacian_value = calculate_laplacian_variance_(cropped_image)

    entropy.append(entropy_value)
    laplacian.append(laplacian_value)

In [219]:
bbox_df['entropy'] = entropy
bbox_df['laplacian'] = laplacian

In [221]:
bbox_df.columns

Index(['image_file', 'bbox', 'bbox_score', 'bbox_id', 'width', 'height',
       'size', 'entropy', 'laplacian'],
      dtype='object')

In [224]:
bbox_df.head()

,image_file,bbox,bbox_score,bbox_id,width,height,size,entropy,laplacian
0,val2017/000000532481.jpg,"[249.8199079291458, 175.21093805640606, 74.004...",0.999274,0,74.004194,55.626326,4116.581369,6.847388,11851.206457
1,val2017/000000532481.jpg,"[284.07836701140275, 370.43747059580454, 13.84...",0.983319,1,13.844298,17.429331,241.296844,6.820085,14014.663049
2,val2017/000000532481.jpg,"[244.05099093550018, 170.61324112294565, 74.56...",0.471152,2,74.563805,73.940382,5513.276229,6.884189,11307.883659
3,val2017/000000532481.jpg,"[285.7852415208732, 370.21400193144893, 10.915...",0.427866,3,10.915034,14.482651,158.078634,6.444467,15793.261786
4,val2017/000000532481.jpg,"[254.66838783379924, 177.22587387050734, 65.52...",0.212189,4,65.529053,58.952817,3863.122282,6.836627,11993.799584


In [ ]:
# bbox_df.to_csv("bbox_df_with_metric.csv", index=False)

In [2]:
# JSON 파일 경로
json_file_path = 'result_keypoints_small_base.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    val_data = json.load(f)

print(f'개수: {len(val_data)}')

개수: 73304


### valid detection과 개수 다름 ***